<a href="https://colab.research.google.com/github/ehdghks316/itwill_studyGroup_D/blob/main/4_28%ED%8C%8C%EC%9D%B4%EC%8D%AC(%EA%B7%9C%EC%A0%9C%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
4.28

import pandas as pd

titanic = pd.read_csv('c:/data/titanic.csv')
titanic.info()
titanic.gender

# 문자 수치화 원핫인코딩
gender_dummy = pd.get_dummies(titanic.gender,prefix='gender')
data = pd.concat([titanic,gender_dummy],axis=1)
data.info()

import statsmodels.api as sm

logit = sm.Logit.from_formula("survived ~ age + fare + sibsp + gender_male", data=data).fit()
logit.summary() # 해석할 줄 알아야함

import numpy as np
# fare
np.exp(0.0159) # odds값
np.log(np.exp(0.0159)) # logit값
(np.exp(0.0159) -1) *100 # 지수함수에 들어가는 값은 coef값(계수의 추정치, 기울기)
요금이 1$ 증가할 때 생존율은 1.6% 증가한다.

# age
(np.exp(-0.0190)-1)*100
나이가 1살 증가할수록 생존가능성은 1.9 감소한다

# sibsp
(np.exp(-0.4630)-1)*100
동승자가 한 명 증가할수록 생존가능성은 37% 감소한다.

# gender_male -> 원핫인코딩한 데이터는 결과가 잘 나오지 않아서  1을 지수함수로 나누고 계산
np.exp(-2.4413) = 0.087
((1/np.exp(-2.4413))-1)*100
남성에 비해 여성의 생존률은 1048% 높다

# 해석할 때 -1하는 이유는 , 100을 곱하는 이유는 백분율로 보기 위해서 
# 표준화로 바꾸고도 해보자
# 정규성으로 잘 안되어 있을 때(왼쪽or오른쪽에 치우친 것) 종속변수에 대해서 로그를 취해서도 해보자

★ 규제신형회귀(Regularized Linear Regression)
#회귀에서 오버피팅은 회귀계수값이 너무크거나 작을 때 발생, 회귀계수를 적절하게 제어하는게 어떨까 그러면 오버피팅을 줄일 수 있지 않을까
- 회귀분석에서 overfitting(과대적합)하게 되면 회귀계수가 크게 설정되며 테스트데이터에 대해서 예측성능이 좋지 않게 된다.
  이를 해결하기 위해서 하나의 방법중에 회귀계수가 기하급수적으로 커지는 것을 제어해야 한다.
 
최적모델을 위한 cost = 잔차(residual) 최소화  + 회귀계수 크기

cost(비용) 목표 = 최소화(RSS[Residual Sum of Square] + alpha * (회귀계수)²)
- alpha는 학습데이터 적합정도와 회귀계수값의 크기를 제어하는 파라미터 
- alpha = 0, 비용함수 최소화(RSS[Residual Sum of Square])
- alpha = 무한(큰값)이라면 RSS에 비해 alpha*(회귀계수)² 값이 커지므로 회귀계수값을 작게 만들어야 비용함수목표에 도달할 수 있다.
- alpha값이 감소하면 RSS 최소화 하는 방향으로 가고
- alpha값이 증가하면 회귀계수를 감소하는 방향으로 간다.
- alpha값으로 패널티를 부여해 회귀계수 값의 크기를 감소시켜 과적합을 개선하는 방식이 규제(Regularization)라고 한다.
- 규제는 L1,L2방식이 있다.
- L2규제는 alpha*(회귀계수)²와 같이 회귀계수의 제곱에 대해 패널티를 부여하는 방식이고 L2규제를 적용한 회귀를 Ridge(릿지)회귀라고 함
- L1규제는 alpha*|회귀계수| 와 같이 회귀계수의 절대값에 대해 패널티를 부여하는 방식이고 L1규제를 적용한 회귀를 Lasso(라쏘)회귀라고 함
  L1규제는 영향력이 크지 않은 회귀계수값은 0으로 변한다. 그러다보니 불필요한 피처들은 제거하고 적절한 피처만 회귀에 포함을 시키는 특징이 있다.
- ElasticNet회귀는 L1,L2규제를 함께 결합한 모델이다.
  
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
boston = pd.read_csv('c:/data/boston.csv')  
boston.info()

x_train = boston.iloc[:,:-1]
y_train = boston.MEDV

lr = LinearRegression()
lr.fit(x_train,y_train)
print('회귀계수(기울기)',lr.coef_)
print('절편',lr.intercept_)
lr.score(x_train,y_train) # 정확도

y_predict = lr.predict(x_train)
rmse = np.sqrt(mean_squared_error(y_train,y_predict)) # rmse, mean_squared_error(y_train,y_predict) : mse
r2_score(y_train,y_predict) # R-squared 설명력
## 데이터를 표준화로 바꾸고 다시 해보기

from sklearn.linear_model import Ridge, Lasso, ElasticNet
# ridge
ridge = Ridge(alpha=10)
ridge.fit(x_train,y_train)
print('회귀계수(기울기)',ridge.coef_)
print('절편',ridge.intercept_)
ridge.score(x_train,y_train) # 정확도
y_predict = ridge.predict(x_train)
rmse = np.sqrt(mean_squared_error(y_train,y_predict)) # rmse, mean_squared_error(y_train,y_predict) : mse
r2_score(y_train,y_predict) # R-squared 설명력

# lasso
lasso = Lasso(alpha=0.1)
lasso.fit(x_train,y_train)
print('회귀계수(기울기)',lasso.coef_)
print('절편',lasso.intercept_)
lasso.score(x_train,y_train) # 정확도
y_predict = lasso.predict(x_train)
rmse = np.sqrt(mean_squared_error(y_train,y_predict)) # rmse, mean_squared_error(y_train,y_predict) : mse
r2_score(y_train,y_predict) # R-squared 설명력

print('사용된 피처의 수',np.sum(lasso.coef_ != 0)) # 12개  영향력이 크지 않은 회귀계수값이 0으로 변해서 
print('사용된 피처의 수',np.sum(ridge.coef_ != 0)) # 13개

# elasticNet
elasticnet = ElasticNet(alpha=0.1)
elasticnet.fit(x_train,y_train)
print('회귀계수(기울기)',elasticnet.coef_)
print('절편',elasticnet.intercept_)
elasticnet.score(x_train,y_train) # 정확도
y_predict = elasticnet.predict(x_train)
rmse = np.sqrt(mean_squared_error(y_train,y_predict)) # rmse, mean_squared_error(y_train,y_predict) : 잔차
r2_score(y_train,y_predict) # R-squared 설명력
print('사용된 피처의 수',np.sum(elasticnet.coef_ != 0)) # 13개

# 컬럼별로 값을 확인하는 방법 
pd.Series(elasticnet.coef_,index=x_train.columns)
pd.Series(lasso.coef_,index=x_train.columns)
pd.Series(ridge.coef_,index=x_train.columns)

# 최적의 alpha값을 구하는 방법
- gridsearchcv, 최적의 파라미터 구하는 방법
from sklearn.model_selection import GridSearchCV

ridge = Ridge()
alphas = [0,0.1,1,10,100]
parameters = {'alpha':alphas}

ridge_cv = GridSearchCV(estimator=ridge, param_grid=parameters,scoring='r2',cv=5)
ridge_cv.fit(x_train,y_train)
ridge_cv.best_params_ # 최적의 파라미터
ridge_cv.best_score_ # 최적의 파라미터의 r2 score값이라고 생각하면 됨

ridge_cv = GridSearchCV(estimator=ridge, param_grid=parameters,scoring='neg_mean_squared_error',cv=5) # mean_squared_error는 작으면 좋은것으로 mean_squared_err사용할 때 여기 옵션에서는 neg_mean_squared_error로 표현(음수), r2는 크면 좋은 것 / 
ridge_cv.fit(x_train,y_train)
ridge_cv.best_params_ # 최적의 파라미터
ridge_cv.best_score_

# 프로젝트 회귀와 규제한 것을 비교해서 기록해놓자


